# Problem Statement

Data overview:

Payments: Each line represents a transaction payment for a contract
• TransactionID -> primary key / unique identifier
• ContractID -> a contract will have multiple repayments
• ClientID -> the client of the contract
• TransactionDate -> This is date in a EPOCH format - you will have to figure out how to convert it
• Amount -> Repayment amount
• Payment Code -> DEFAULT means a payment was not made

Clients: Each line represents a unique client and their information
• ClientID
• Entity Type -> Business type
• Entity Year Established -> First year the business was open

Business question:
Payment defaults are detrimental to the business and are a significant cost factor.
Are there any key trends in the data which can help me avoid default-prone customers in the future?

In [1]:
import pandas as pd
from datetime import datetime

# First Look

In [2]:
# Load the data files
clients_df_org = pd.read_csv('data/Clients.csv')
payments_df_org = pd.read_csv('data/Payments.csv')

In [3]:
payments_df_org

,transaction_id,contract_id,client_id,transaction_date,payment_amt,payment_code
0,20175,927,1,1527012511,66.66,PAYMENT
1,8485,927,1,1511716095,66.66,PAYMENT
2,13778,927,1,1519319303,66.66,PAYMENT
3,22768,927,1,1529863724,66.66,PAYMENT
4,15698,927,1,1521738504,66.66,PAYMENT
...,...,...,...,...,...,...
25554,25075,1603,1280,1532023764,1666.68,PAYMENT
25555,24711,1603,1280,1531764560,0.01,PAYMENT
25556,25076,1603,1280,1532023764,64.99,PAYMENT
25557,25132,1627,1281,1532282886,0.01,PAYMENT


In [4]:
# a function to check data problems
def check_data(df):
    print(f"Info:\n{df.info()}")
    print(f"Number of null values for each col:\n{df.isnull().sum()}")
    print(f"Number of duplicates:\n{df.duplicated().sum()}")
    print(f"Number of unique values for each col: {df.nunique()}")

In [5]:
check_data(payments_df_org)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25559 entries, 0 to 25558
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    25559 non-null  int64  
 1   contract_id       25559 non-null  int64  
 2   client_id         25559 non-null  int64  
 3   transaction_date  25559 non-null  int64  
 4   payment_amt       25559 non-null  float64
 5   payment_code      25559 non-null  object 
dtypes: float64(1), int64(4), object(1)
memory usage: 1.2+ MB
Info:
None
Number of null values for each col:
transaction_id      0
contract_id         0
client_id           0
transaction_date    0
payment_amt         0
payment_code        0
dtype: int64
Number of duplicates:
0
Number of unique values for each col: transaction_id      25559
contract_id          1643
client_id            1281
transaction_date      931
payment_amt           906
payment_code            2
dtype: int64


The payments data seems clean, with no duplicate and missing value. The only problem is the transaction_date column, which is in epoch format.

Note that there are 1281 different clients who made payment. Let's cross check it with the customers data.

In [6]:
check_data(clients_df_org)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1287 entries, 0 to 1286
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   client_id                1287 non-null   int64 
 1   entity_type              1287 non-null   object
 2   entity_year_established  1287 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 30.3+ KB
Info:
None
Number of null values for each col:
client_id                  0
entity_type                0
entity_year_established    0
dtype: int64
Number of duplicates:
0
Number of unique values for each col: client_id                  1281
entity_type                  10
entity_year_established      20
dtype: int64


In [7]:
# find rows with the same client_id in Clients
clients_duplicates = clients_df_org[clients_df_org.duplicated(subset=['client_id'], keep=False)]
print(f"Duplicates in Clients:\n{clients_duplicates}")

Duplicates in Clients:
      client_id                     entity_type  entity_year_established
245         591      Australian Private Company                     2013
306         591      Australian Private Company                     2007
332         473      Australian Private Company                     2016
350         473          Individual/Sole Trader                     2008
401         165      Australian Private Company                     2015
816         591      Australian Private Company                     2015
1063        797  Discretionary Investment Trust                     2016
1115        165          Individual/Sole Trader                     2006
1172       1262      Australian Private Company                     2005
1182       1262      Australian Private Company                     2016
1275        797      Australian Private Company                     2002


There are 5 clients with the same ID yet different entity type and/or year of establishment

- Companies with the Same Client ID but Different Entity Year Established: These are companies that might have the same Entity Type but are listed with different years of establishment. This could happen if the business was registered multiple times under different legal frameworks or if the data is entered incorrectly.

- Companies with Different Entity Type and Entity Year Established: These are companies that have changed both their legal structure (i.e., Entity Type) and their registration year (Entity Year Established), likely reflecting a reorganization. For companies who change to another entity type, their historical record is still kept. 

We would further examine these clients after cleaning the data.

# Data Preprocessing

## Data Cleaning

In [8]:
payments_df = payments_df_org.copy()
clients_df = clients_df_org.copy()

In [9]:
# convert the "transaction_date" column from epoch to datetime
payments_df['transaction_datetime'] = pd.to_datetime(payments_df_org['transaction_date'], unit='s')
payments_df = payments_df.drop(columns=['transaction_date'])
payments_df['transaction_datetime']

0       2018-05-22 18:08:31
1       2017-11-26 17:08:15
2       2018-02-22 17:08:23
3       2018-06-24 18:08:44
4       2018-03-22 17:08:24
                ...        
25554   2018-07-19 18:09:24
25555   2018-07-16 18:09:20
25556   2018-07-19 18:09:24
25557   2018-07-22 18:08:06
25558   2018-07-22 18:08:06
Name: transaction_datetime, Length: 25559, dtype: datetime64[ns]

Now to determine how to deal with the duplicated client ids, we will examine their payment data

In [10]:
payments_duplicates = payments_df[payments_df['client_id'].isin(clients_duplicates['client_id'])]
# check if the payment code is default for the duplicated clients
payments_duplicates[payments_duplicates['payment_code'] == 'default']

,transaction_id,contract_id,client_id,payment_amt,payment_code,transaction_datetime


**None of these clients have default payment.**

Plus, the earliest recorded payment in the dataset is 2017, which is after all the recorded established dates for the duplicated clients. Thus, it might be suitable to only keep the record with establishment year nearest to the first recorded payment of these clients.

In [11]:
payments_df['transaction_datetime'].min(), payments_df['transaction_datetime'].max()

(Timestamp('2017-07-02 18:08:02'), Timestamp('2018-07-24 18:08:10'))

In [12]:
# Find the record with the most recent established year for each client
filtered_clients = clients_df.loc[clients_duplicates.groupby('client_id')['entity_year_established'].idxmax()]

clients_df = clients_df[~clients_df['client_id'].isin(clients_duplicates['client_id'])]
clients_df = pd.concat([clients_df, filtered_clients])

print(clients_df.duplicated(subset=['client_id']).sum()) # should be 0
print(clients_df.info())

0
<class 'pandas.core.frame.DataFrame'>
Index: 1281 entries, 0 to 1182
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   client_id                1281 non-null   int64 
 1   entity_type              1281 non-null   object
 2   entity_year_established  1281 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 40.0+ KB
None


## Feature Engineering

Some features that would be useful in determining the default-prone customers:

- Time of transaction: day in month, month in year
- Clients' entity types and number of years established
- Clients' payment history: total amount paid, average amount per transaction, number of previous payments, frequency of payment, number of previous contract
- Clients' default history: If they defaulted previously

In [13]:
# calculate the current age of the client = current year - year of birth
clients_df["entity_age"] = datetime.now().year - clients_df["entity_year_established"]
clients_df["entity_age"].describe()

count    1281.000000
mean       15.917252
std         5.715332
min         7.000000
25%        11.000000
50%        15.000000
75%        21.000000
max        26.000000
Name: entity_age, dtype: float64

In [14]:
# sort by client_id and payment_date to ensure we're processing payments in chronological order
payments_df = payments_df.sort_values(by=['client_id', 'transaction_datetime', 'transaction_id'])

aggregated_data = []

# loop through each payment to calculate historical data before the payment date
for _, row in payments_df.iterrows():
    client_id = row['client_id']
    transaction_datetime = row['transaction_datetime']
    transaction_id = row['transaction_id']
    payment_code = row['payment_code']
    
    previous_payments = payments_df[(payments_df['client_id'] == client_id) & (payments_df['transaction_datetime'] <= transaction_datetime) & (payments_df['transaction_id'] < transaction_id) & (payments_df['payment_code'] != "DEFAULT")]
    previous_defaults = payments_df[(payments_df['client_id'] == client_id) & (payments_df['transaction_datetime'] <= transaction_datetime) & (payments_df['transaction_id'] < transaction_id) & (payments_df['payment_code'] == "DEFAULT")]
    
    # aggregate previous payment details
    total_amount_paid_before = previous_payments['payment_amt'].sum() if not previous_payments.empty else 0
    avg_amount_per_transaction_before = previous_payments['payment_amt'].mean() if not previous_payments.empty else 0
    num_payments_before = previous_payments.shape[0] if not previous_payments.empty else 0
    num_contracts_before = previous_payments['contract_id'].nunique()
    num_defaults_before = previous_defaults.shape[0] if not previous_defaults.empty else 0

    aggregated_data.append({
        'client_id': client_id,
        'transaction_id': transaction_id,
        'transaction_datetime': transaction_datetime,
        'total_amount_paid_before': round(total_amount_paid_before, 2),
        'avg_amount_per_transaction_before': round(avg_amount_per_transaction_before, 2),
        'num_payments_before': num_payments_before,
        'num_contracts_before': num_contracts_before,
        'num_defaults_before': num_defaults_before,
    })

client_payment_history_df = pd.DataFrame(aggregated_data)

# # If you want to aggregate these into one row per client, just group by client_id and aggregate
# client_payment_history_df = aggregated_df.groupby('client_id').agg(
#     total_amount_paid=('total_amount_paid', 'sum'),
#     avg_amount_per_transaction=('avg_amount_per_transaction', 'mean'),
#     payment_frequency=('payment_frequency', 'sum'),
#     num_contracts_before=('num_contracts_before', 'sum')
# ).reset_index()

# # Round results for cleaner output
# client_payment_history_df['total_amount_paid'] = client_payment_history_df['total_amount_paid'].round(2)
# client_payment_history_df['avg_amount_per_transaction'] = client_payment_history_df['avg_amount_per_transaction'].round(2)

# client_payment_history_df
client_payment_history_df

,client_id,transaction_id,transaction_datetime,total_amount_paid_before,avg_amount_per_transaction_before,num_payments_before,num_contracts_before,num_defaults_before
0,1,8322,2017-11-22 17:08:16,0.00,0.00,0,0,0
1,1,8323,2017-11-22 17:08:16,5050.04,5050.04,1,1,0
2,1,8484,2017-11-26 17:08:15,5100.04,2550.02,2,1,0
3,1,8485,2017-11-26 17:08:15,5566.71,1855.57,3,1,0
4,1,10125,2017-12-21 17:08:22,5633.37,1408.34,4,1,0
...,...,...,...,...,...,...,...,...
25554,1280,24711,2018-07-16 18:09:20,20000.05,20000.05,1,1,0
25555,1280,25075,2018-07-19 18:09:24,20000.06,10000.03,2,1,0
25556,1280,25076,2018-07-19 18:09:24,21666.74,7222.25,3,1,0
25557,1281,25131,2018-07-22 18:08:06,0.00,0.00,0,0,0


In [15]:
client_payment_history_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25559 entries, 0 to 25558
Data columns (total 8 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   client_id                          25559 non-null  int64         
 1   transaction_id                     25559 non-null  int64         
 2   transaction_datetime               25559 non-null  datetime64[ns]
 3   total_amount_paid_before           25559 non-null  float64       
 4   avg_amount_per_transaction_before  25559 non-null  float64       
 5   num_payments_before                25559 non-null  int64         
 6   num_contracts_before               25559 non-null  int64         
 7   num_defaults_before                25559 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(5)
memory usage: 1.6 MB


## Data Merging

In [16]:
# merge the payment history with the payments_df
payments_df = payments_df.merge(client_payment_history_df, on=['client_id', 'transaction_id', 'transaction_datetime'])

In [17]:
# merge the payment_df with the clients_df, inner join since we generally want to keep only the clients that have made payments
merged_df = payments_df.merge(clients_df, on='client_id', how='inner')

In [18]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25559 entries, 0 to 25558
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   transaction_id                     25559 non-null  int64         
 1   contract_id                        25559 non-null  int64         
 2   client_id                          25559 non-null  int64         
 3   payment_amt                        25559 non-null  float64       
 4   payment_code                       25559 non-null  object        
 5   transaction_datetime               25559 non-null  datetime64[ns]
 6   total_amount_paid_before           25559 non-null  float64       
 7   avg_amount_per_transaction_before  25559 non-null  float64       
 8   num_payments_before                25559 non-null  int64         
 9   num_contracts_before               25559 non-null  int64         
 10  num_defaults_before               

In [19]:
# save merged_df to a CSV file
merged_df.to_csv('data/ClientsPayments.csv', index=False)